In [1]:
import numpy as np

In [49]:
class Operator:
    t1 = -1
    t2 = -1

    def __init__(self, t1, t2):
        self.t1 = t1
        self.t2 = t2

class State:

    towers = -1
    towerpos = dict()
    previous = None
    gval = 0
    n = -1

    def __init__(self, n, prevtowers = None, prevtowerpos = None):
        self.towers = np.zeros([4, n])
        self.n = n
        self.towers[0] = [i for i in range(1, n+1)]
        self.towerpos = {0: 0, 1 : self.n, 2: self.n, 3: self.n}
        
        if prevtowers is not None:
            self.towers = np.zeros([4, n])
            for i in range(4):
                for j in range(n):
                    self.towers[i, j] = prevtowers[i][j]
            
        if prevtowerpos is not None:
            for key in prevtowerpos:
                self.towerpos[key] = prevtowerpos[key]

    def isEqual(self, s):
        for i in range(4):
            for j in range(self.n):
                if (self.towers[i][j] != s.towers[i][j]):
                    return 0
        return 1
    
    def applyOperator(self, o):
        s1 = State(self.n, self.towers, self.towerpos)
        s1.previous = self
        s1.gval = self.gval + 1
        
        if self.towerpos[o.t1] < self.n:
            k = self.towers[o.t1, self.towerpos[o.t1]]

            if self.towerpos[o.t2] == self.n:
                s1.towers[o.t2, self.towerpos[o.t2] - 1] = k
                s1.towers[o.t1, self.towerpos[o.t1]] = 0

                s1.towerpos[o.t1] += 1
                s1.towerpos[o.t2] -= 1

            elif (self.towers[o.t2, self.towerpos[o.t2]] > k):
                s1.towers[o.t2, self.towerpos[o.t2] - 1] = k
                s1.towers[o.t1, self.towerpos[o.t1]] = 0

                s1.towerpos[o.t1] += 1
                s1.towerpos[o.t2] -= 1
        
        return s1

    def printState(self):
        for i, char in enumerate(["A", "B", "C", "D"]):
            print(f"Tower {char}: {self.towers[i]}")

In [54]:
#Function that indicates whether a state is present in a list
def isPresentStateInList(state, searchList):
    for elem in searchList:
        if(state.isEqual(elem) == 1):
            return 1

    return 0

#Function that indicates whether a state is present in a priority list
def isPresentStateInPriorityList(state, searchList):
    for [elem, dist] in searchList:
        if(state.isEqual(elem) == 1):
            return 1
        
    return 0

def insertStateInPriorityQueue(searchList, state, distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(distanceToGoal < searchList[i][1]):
            index = i
            break
    
    if(len(searchList) == 0 or index == -1):
        searchList.append([state,distanceToGoal])
    else:
        searchList.insert(index, [state,distanceToGoal])

#Reinserts the element in priority queue if the new value is less than the 
#value present in queue.
def checkAndUpdateStateInPriorityQueue(searchList,state,distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(state.isEqual(searchList[i][0])):
            if distanceToGoal < searchList[i][1]:
                    index = i
            break
            
    if index!=-1:
        searchList.remove([searchList[index][0],searchList[index][1]])
        insertStateInPriorityQueue(searchList, state, distanceToGoal)
        

        
        
def retrievePathFromState(state):
    visitedstatelist = []
    visitedstatelist.append(state)
    # state.printState()
    prev = state.previous
    # prev.printState()
    counter = 0
    while(prev != None):
        visitedstatelist.append(prev)
        #prev.printState()
        prev = prev.previous
        counter +=1
    visitedstatelist.reverse()
    for i in range(len(visitedstatelist)):
        print(f"Step {i}")
        visitedstatelist[i].printState()
        pass
    print("Size of path is ",counter,state.gval)

In [4]:
def h(state):
    zero_count = 0
    for i in state.towers[3]:
        if i == 0:
            zero_count += 1
    return zero_count

In [106]:
def searchDFS(start, goal):
    openList = []   # Frontier list
    closedList = [] #Explored list
    
    openList.append(start)
    p = 0
    
    while(len(openList) != 0):
        p += 1
        print(f"STEP {p}")
        print("-" * 40)
        print("OPEN LIST")
        for i in openList:
            i.printState()
            print("-" * 20)

        first = openList[0]
        openList.remove(first)
        closedList.append(first)

        print("POP")
        first.printState()
        print("-" * 20)
        print("NODES ADDED")
        
        for i in range(4): # For loop to generate all successors
            for j in range(4):
                if i == j:
                    continue
                o = Operator(i, j)
                succState = first.applyOperator(o)
                
                if(succState.isEqual(goal) == 1):
                    retrievePathFromState(succState)
                    print(f"No of nodes explored = {p}")
                    return
                
                if(isPresentStateInList(succState, openList) == 0 and isPresentStateInList(succState,closedList) == 0):
                    openList.insert(0, succState)

                    succState.printState()
                    print("-"*10)
        
        print("-"*40)
        if p == 3:
            break
            

n = 3
start = State(n)
goal = [[0 for i in range(n)] for i in range(3)]
goal.append([i+1 for i in range(n)])
goal = State(n, goal)
searchDFS(start, goal)

STEP 1
----------------------------------------
OPEN LIST
Tower A: [1. 2. 3.]
Tower B: [0. 0. 0.]
Tower C: [0. 0. 0.]
Tower D: [0. 0. 0.]
--------------------
POP
Tower A: [1. 2. 3.]
Tower B: [0. 0. 0.]
Tower C: [0. 0. 0.]
Tower D: [0. 0. 0.]
--------------------
NODES ADDED
Tower A: [0. 2. 3.]
Tower B: [0. 0. 1.]
Tower C: [0. 0. 0.]
Tower D: [0. 0. 0.]
----------
Tower A: [0. 2. 3.]
Tower B: [0. 0. 0.]
Tower C: [0. 0. 1.]
Tower D: [0. 0. 0.]
----------
Tower A: [0. 2. 3.]
Tower B: [0. 0. 0.]
Tower C: [0. 0. 0.]
Tower D: [0. 0. 1.]
----------
----------------------------------------
STEP 2
----------------------------------------
OPEN LIST
Tower A: [0. 2. 3.]
Tower B: [0. 0. 0.]
Tower C: [0. 0. 0.]
Tower D: [0. 0. 1.]
--------------------
Tower A: [0. 2. 3.]
Tower B: [0. 0. 0.]
Tower C: [0. 0. 1.]
Tower D: [0. 0. 0.]
--------------------
Tower A: [0. 2. 3.]
Tower B: [0. 0. 1.]
Tower C: [0. 0. 0.]
Tower D: [0. 0. 0.]
--------------------
POP
Tower A: [0. 2. 3.]
Tower B: [0. 0. 0.]
Tower

In [92]:
def searchAStar(start, goal):
    openList = []   # Frontier list
    closedList = [] #Explored list
    
    insertStateInPriorityQueue(openList, start, h(start) + start.gval)
    p = 0

    while(len(openList) != 0):
        first_el = openList[0]
        first = first_el[0]
        openList.remove(first_el)
        insertStateInPriorityQueue(closedList, first_el[0], first_el[1])
        p += 1
        
        for i in range(4): # For loop to generate all successors
            for j in range(4):
                if i == j:
                    continue
                o = Operator(i, j)
                succState = first.applyOperator(o)
                if(succState.isEqual(goal) == 1):
                    retrievePathFromState(succState)
                    print(f"No of nodes explored = {p}")
                    return
                
                if isPresentStateInPriorityList(succState, openList) == 0:
                    if isPresentStateInPriorityList(succState,closedList) == 0:
                        insertStateInPriorityQueue(openList, succState, h(succState) + succState.gval)
                else:
                    checkAndUpdateStateInPriorityQueue(openList, succState, h(succState) + succState.gval)
                
                if isPresentStateInPriorityList(succState, closedList) == 1:
                    checkAndUpdateStateInPriorityQueue(closedList, succState, h(succState) + succState.gval)
    
n = 3
start = State(n)
goal = [[0 for i in range(n)] for i in range(3)]
goal.append([i+1 for i in range(n)])
goal = State(n, goal)               
searchAStar(start, goal)

Step 0
Tower A: [1. 2. 3.]
Tower B: [0. 0. 0.]
Tower C: [0. 0. 0.]
Tower D: [0. 0. 0.]
Step 1
Tower A: [0. 2. 3.]
Tower B: [0. 0. 1.]
Tower C: [0. 0. 0.]
Tower D: [0. 0. 0.]
Step 2
Tower A: [0. 0. 3.]
Tower B: [0. 0. 1.]
Tower C: [0. 0. 2.]
Tower D: [0. 0. 0.]
Step 3
Tower A: [0. 0. 0.]
Tower B: [0. 0. 1.]
Tower C: [0. 0. 2.]
Tower D: [0. 0. 3.]
Step 4
Tower A: [0. 0. 0.]
Tower B: [0. 0. 1.]
Tower C: [0. 0. 0.]
Tower D: [0. 2. 3.]
Step 5
Tower A: [0. 0. 0.]
Tower B: [0. 0. 0.]
Tower C: [0. 0. 0.]
Tower D: [1. 2. 3.]
Size of path is  5 5
No of nodes explored = 22
